In [1]:
import sys
import os
import importlib
import time
import random
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer #计算tfidf
from sklearn.feature_extraction.text import CountVectorizer  #计算df
from sklearn.feature_extraction.text import TfidfVectorizer  #“一步到位”
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model.logistic import LogisticRegression
from sklearn.ensemble import RandomForestClassifier  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.externals import joblib
from sklearn import metrics
from sklearn.model_selection import  GridSearchCV

id_num = []
word_article = []
words_article = []
label = []
X_train = []
y_train = []
X_test = []
y_test = []

load_start = time.time()
importlib.reload(sys)
with open('train_set.csv','r',encoding = 'utf-8') as f:
    lines = f.readlines()
    random.shuffle(lines)
    for line in lines[1:15000]:
        line = line.split(',')
        X_train.append(line[2])
        y_train.append(line[3])
    for line in lines[100000:]:
        line = line.split(',')
        X_test.append(line[2])
        y_test.append(line[3])
print("Time consumption on loading:",time.time() - load_start)


train_start = time.time()
step = [
            ('vect',CountVectorizer(max_df=0.9, min_df=3)),
            ('tfidf',TfidfTransformer()),
#             ('pca',PCA(n_components=3000)),
#             ('clf',RandomForestClassifier()),
#             ('clf',svm.SVC(C=1,kernel='linear'))
            ('clf',svm.LinearSVC(C=1))
#             ('clf',SGDClassifier(loss="modified_huber", penalty="elasticnet", alpha=0.00001, n_jobs=-1))
#             ('clf',tree.DecisionTreeClassifier())
        ]
ppl_clf = Pipeline(step)
parameters = {
    'vect__ngram_range': [ (1, 2), (1, 3)], #'vect__max_df': (0.9, 0.95),'vect__min_df': (3, 10),
#     'tfidf__use_idf': (True, False),#'tfidf__norm': ('l1', 'l2'),
#     'clf__alpha': (0.00001, 0.000001), 'clf__n_iter': (10, 50) 
}

#GridSearchCV用于寻找vectorizer词频统计, tfidftransformer特征变换和分类模型的最优参数
grid_search = GridSearchCV(ppl_clf, parameters,)
grid_search.fit(X_train,y_train) #遍历执行候选参数，寻找最优参数
best_parameters = dict(grid_search.best_estimator_.get_params())#get实例中的最优参数
for param_name in sorted(parameters.keys()):
    print("BESTTTT：\t%s: %r" % (param_name, best_parameters[param_name])),'\n'#输出参数结果

ppl_clf.fit(X_train, y_train) #将ppl_clf实例中的参数重写为最优结果
# joblib.dump(ppl_clf, 'lineSVM_20000__.m')
print("Time consumption on training:",time.time() - train_start)


test_start = time.time()
prediction = ppl_clf.predict(X_train) #训练集上准确率
print(metrics.classification_report(y_train,prediction))
prediction = ppl_clf.predict(X_test) #测试集上准确率
print(metrics.classification_report(y_test,prediction))
print("Time consumption on testing:",time.time() - test_start)

Time consumption on loading: 2.016610860824585
BESTTTT：	vect__ngram_range: (1, 2)
Time consumption on training: 636.975100517273
             precision    recall  f1-score   support

         1
       0.99      0.98      0.98       773
        10
       0.99      0.99      0.99       720
        11
       0.99      0.99      0.99       539
        12
       1.00      1.00      1.00       775
        13
       0.99      1.00      0.99      1216
        14
       1.00      0.99      0.99       988
        15
       1.00      1.00      1.00      1122
        16
       0.99      0.98      0.98       469
        17
       0.99      0.99      0.99       454
        18
       1.00      1.00      1.00      1054
        19
       1.00      0.99      0.99       806
         2
       0.99      1.00      1.00       408
         3
       0.99      1.00      1.00      1268
         4
       1.00      1.00      1.00       538
         5
       1.00      0.99      1.00       330
         6
       1.00